In [1]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps


In [2]:
#-------------------
# importing libraries
#-------------------
import tensorflow as tf
import efficientnet.tfkeras as efn
import tensorflow_addons as tfa

import pandas as pd
import numpy as np

import os
import shutil

In [3]:
HEIGHT,WIDTH = 224,224
CHANNELS = 3
BATCH_SIZE = 32
TEST_PATH = "../input/pneumonia-classification-challenge/pneumonia_dataset/pneumonia_dataset/test/"
test_df = pd.read_csv("../input/pneumonia-classification-challenge/pneumonia_dataset/pneumonia_dataset/test.csv")
classes_dict = {
                 0 : 'normal', 
                 1 : 'pneumonia'
               }
AUTO = tf.data.experimental.AUTOTUNE


MODEL_PATH = "../input/pneumonia-classification-challenge-training"

In [4]:

def process_img(filepath):
    image = tf.io.read_file(filepath)
    image = tf.image.decode_png(image, channels=CHANNELS)
    image = tf.image.convert_image_dtype(image, tf.float32) 
    image = tf.image.resize(image, [HEIGHT,WIDTH])
    return image


files_ls = list(test_df["filename"])
files_ls = [TEST_PATH + i for i in files_ls]

test_ds = tf.data.Dataset.from_tensor_slices(files_ls)
test_ds = test_ds.map(process_img,num_parallel_calls=AUTO)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTO)

In [5]:
pred_arr = 0 
count = 0

for i,file in enumerate(os.listdir(MODEL_PATH)):
    if file.split(".")[-1] == "h5":
        count+=1
        model = tf.keras.models.load_model(os.path.join(MODEL_PATH,file))
        pred_arr += model.predict(test_ds)

pred_arr = pred_arr/count 

#{'normal': 0, 'pneumonia': 1}
pred_ls = [classes_dict[np.argmax(i)] for i in pred_arr]
test_df["label"] = pred_ls
test_df.to_csv("submission.csv",index = False)
test_df.head()

,filename,label
0,CXR_test_519.png,pneumonia
1,CXR_test_578.png,normal
2,CXR_test_359.png,normal
3,CXR_test_573.png,pneumonia
4,CXR_test_471.png,pneumonia
